**Model Run**

In this file, the model run as well as sensitivity analysis are conducted:

1. Initial Model Run
2. Sensitivity Analysis 
3. Experiment

In [1]:
# import necessary libraries and model
from model import AdaptationModel
import matplotlib.pyplot as plt
import networkx as nx
from collections import defaultdict 
import pandas as pd

In [2]:
# set-up model parameters
# run time: 100 years --> 400 quarters, so 400 ticks in total
run_length = 400
# set the number of replications 
replication_number = 10
# set the number of households
number_of_households = 500

**Initial Run**

In [3]:
# use defaultdict to store the results with different seeds
model_results =  defaultdict(list)
agent_results = defaultdict(list)
# Run the model based on the parameters above, no subsidy included
for i in range(replication_number):
    seed = i  # set the seed
    model = AdaptationModel(number_of_households= number_of_households,subsidy_rate=0, income_threshold=2000, flood_map_choice="harvey", network="no_network", seed=seed)
    for tick in range(run_length):
        model.step() 
    model_data = model.datacollector.get_model_vars_dataframe()
    agent_data = model.datacollector.get_agent_vars_dataframe()
    model_results[i].append(model_data)
    agent_results[i].append(agent_data)
        

Agent 1 moved in POINT (252321.5871971925 3340705.513117809) with income 4581 and savings 4581 and age 74
Agent 2 moved in POINT (301878.00107236515 3292798.1622913997) with income 2096 and savings 2096 and age 39
Agent 3 moved in POINT (247224.45927744108 3293410.000524047) with income 1391 and savings 2782 and age 66
Agent 3 is eligible for subsidy
Agent 4 moved in POINT (303256.2160899186 3270862.306458518) with income 23314 and savings 46628 and age 60
Agent 5 moved in POINT (236273.2670061881 3294321.0790699003) with income 4734 and savings 14202 and age 73
Agent 6 moved in POINT (243862.10181031728 3312803.9086203706) with income 1262 and savings 1262 and age 75
Agent 6 is eligible for subsidy
Agent 7 moved in POINT (270528.6095320853 3306436.6782355327) with income 5853 and savings 11706 and age 79
Agent 8 moved in POINT (218849.33923082173 3330077.757788037) with income 3104 and savings 3104 and age 58
Agent 9 moved in POINT (239434.95173576628 3343520.3857595697) with income 1

**Save the Results**

In [11]:
def convert_to_dataframe(results, replication_number):
    """Convert the results to a dataframe, single index"""
    dfs = [pd.concat(results[i], keys=[i], names=['replication/seed']) for i in range(replication_number)]
    dataframe = pd.concat(dfs)
    dataframe.reset_index(inplace=True)
    dataframe.rename(columns={'level_1': 'Step'}, inplace=True)
    return dataframe

In [5]:
# convert the model results to dataframe
model_dataframe = convert_to_dataframe(model_results, replication_number)
# convert the agent results to dataframe
agent_dataframe = convert_to_dataframe(agent_results, replication_number)

In [87]:
# Save the results to csv files
model_dataframe.to_csv(f"../results/{'model_results_no_subsidy'}.csv", index=False)
agent_dataframe.to_csv(f"../results/{'agent_results_no_subsidy'}.csv", index=False)

In [62]:
# Load the data with the following code
# change the file name to the file you want to load
#model_dataframe = pd.read_csv(f"../results/{'model_results'}.csv")

**Sensitivity Analysis**

In [3]:
# Parameters for the sensitivity analysis
subsidy_rate = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 
                0.7, 0.8, 0.9, 1]   # subsidy rate offered
income_threshold = [0, 1000, 3000, 5000, 8000, 10000] # agents which have lower income then the threshold eligible for subsidy
saving_threshold = [0, 0.1, 0.25, 0.5, 0.75] # agent calculate_saving(self) threshold, for saving or consuming
flood_probabilities = [0.02, 0.07, 0.1, 0.3, 0.5, 0.7] # different flood probabilities for harvey map

**1. Saving Threshold Sensitivity**

In [12]:
# use defaultdict to store the results with different seeds
model_results_saving_threshold =  defaultdict(list)
agent_results_saving_threshold = defaultdict(list)
# Run the sensitivity for different saving thresholds, no subsidy included
for rate in saving_threshold:
    for i in range(replication_number):
        seed = i  # set the seed
        model = AdaptationModel(number_of_households= number_of_households,subsidy_rate=0, income_threshold=2000, saving_threshold=rate, flood_map_choice="harvey", network="no_network", seed=seed)
        for tick in range(run_length):
            model.step() 
        model_data2 = model.datacollector.get_model_vars_dataframe()
        agent_data2 = model.datacollector.get_agent_vars_dataframe()
        agent_data2.reset_index(inplace=True)
        model_results_saving_threshold[i,rate].append(model_data2)
        agent_results_saving_threshold[i,rate].append(agent_data2)

**Save the Results**

In [13]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs = {}

# Concatenate the dataframes for each (i, rate) key
for key, result_list in model_results_saving_threshold.items():
    i, rate = key
    concatenated_dfs[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
model_dataframe_saving_threshold = pd.concat(concatenated_dfs.values(), keys=concatenated_dfs.keys(), names=['replication/seed', 'rate', 'Step'])

# Reset the index 
model_dataframe_saving_threshold.reset_index(inplace=True)


In [14]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs2 = {}

# Concatenate the dataframes for each (i, rate) key
for key, result_list in agent_results_saving_threshold.items():
    i, rate = key
    concatenated_dfs2[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
agent_dataframe_saving_threshold = pd.concat(concatenated_dfs2.values(), keys=concatenated_dfs2.keys(), names=['replication/seed', 'rate', 'index'])

# Reset the index 
agent_dataframe_saving_threshold.reset_index(inplace=True)
# drop index column
agent_dataframe_saving_threshold.drop(columns=['index'], inplace=True)


In [97]:
# Save the results to csv files
model_dataframe_saving_threshold.to_csv(f"../results/{'model_sensitivity_results_saving_threshold'}.csv", index=False)
agent_dataframe_saving_threshold.to_csv(f"../results/{'agent_sensitivity_results_saving_threshold'}.csv", index=False)

In [100]:
#upload the data with the following code
# change the file name to the file you want to load
#model_dataframe_saving_threshold = pd.read_csv(f"../results/{'model_sensitivity_results_saving_threshold'}.csv")

**2.Income Threshold**

In [18]:
# use defaultdict to store the results with different seeds
model_results_income_threshold =  defaultdict(list)
agent_results_income_threshold = defaultdict(list)
# Run the sensitivity for different income thresholds, when subsidy is 10% and saving threshold 0.25
for income in income_threshold:
    for i in range(replication_number):
        seed = i  # set the seed
        model = AdaptationModel(number_of_households= number_of_households,subsidy_rate=0.1, income_threshold=income, saving_threshold=0.25, flood_map_choice="harvey", network="no_network", seed=seed)
        for tick in range(run_length):
            model.step() 
        model_data3 = model.datacollector.get_model_vars_dataframe()
        agent_data3 = model.datacollector.get_agent_vars_dataframe()
        agent_data3.reset_index(inplace=True)
        model_results_income_threshold[i,income].append(model_data3)
        agent_results_income_threshold[i,income].append(agent_data3)

**Save the Results**

In [21]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs = {}

# Concatenate the dataframes for each key
for key, result_list in model_results_income_threshold.items():
    i, income = key
    concatenated_dfs[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
model_dataframe_income_threshold = pd.concat(concatenated_dfs.values(), keys=concatenated_dfs.keys(), names=['replication/seed', 'income_threshold', 'Step'])

# Reset the index 
model_dataframe_income_threshold.reset_index(inplace=True)



In [22]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs2 = {}

# Concatenate the dataframes for each (i, rate) key
for key, result_list in agent_results_income_threshold.items():
    i, income = key
    concatenated_dfs2[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
agent_dataframe_income_threshold = pd.concat(concatenated_dfs2.values(), keys=concatenated_dfs2.keys(), names=['replication/seed', 'income_threshold', 'index'])

# Reset the index 
agent_dataframe_income_threshold.reset_index(inplace=True)
# drop index column
agent_dataframe_income_threshold.drop(columns=['index'], inplace=True)


C:\Users\canan\AppData\Local\Temp\ipykernel_26396\1200854002.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  agent_dataframe_income_threshold = pd.concat(concatenated_dfs2.values(), keys=concatenated_dfs2.keys(), names=['replication/seed', 'income_threshold', 'index'])


In [35]:
# save the results to csv files
model_dataframe_income_threshold.to_csv(f"../results/{'model_sensitivity_results_income_threshold'}.csv", index=False)
agent_dataframe_income_threshold.to_csv(f"../results/{'agent_sensitivity_results_income_threshold'}.csv", index=False)

In [38]:
# upload the data with the following code
# change the file name to the file you want to load
# model_dataframe_income_threshold = pd.read_csv(f"../results/{'model_sensitivity_results_income_threshold'}.csv")

**3.Subsidy Rate**

In [16]:
# use defaultdict to store the results with different seeds
model_results_subsidy_rate =  defaultdict(list)
agent_results_subsidy_rate = defaultdict(list)
# Run the sensitivity for different income thresholds, when income is 3000 and saving threshold 0.25
for subsidy in subsidy_rate:
    for i in range(replication_number):
        seed = i  # set the seed
        model = AdaptationModel(number_of_households= number_of_households,subsidy_rate=subsidy, income_threshold=3000, saving_threshold=0.25, flood_map_choice="harvey", network="no_network", seed=seed)
        for tick in range(run_length):
            model.step() 
        model_data4 = model.datacollector.get_model_vars_dataframe()
        agent_data4 = model.datacollector.get_agent_vars_dataframe()
        agent_data4.reset_index(inplace=True)
        model_results_subsidy_rate[i,subsidy].append(model_data4)
        agent_results_subsidy_rate[i,subsidy].append(agent_data4)

**Save the Results**

In [17]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs = {}

# Concatenate the dataframes for each key
for key, result_list in model_results_subsidy_rate.items():
    i, subsidy = key
    concatenated_dfs[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
model_dataframe_subsidy_rate = pd.concat(concatenated_dfs.values(), keys=concatenated_dfs.keys(), names=['replication/seed', 'subsidy_rate', 'Step'])

# Reset the index 
model_dataframe_subsidy_rate.reset_index(inplace=True)



In [18]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs2 = {}

# Concatenate the dataframes for each key
for key, result_list in agent_results_subsidy_rate.items():
    i, subsidy = key
    concatenated_dfs2[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
agent_dataframe_subsidy_rate = pd.concat(concatenated_dfs2.values(), keys=concatenated_dfs2.keys(), names=['replication/seed', 'subsidy_rate', 'index'])

# Reset the index 
agent_dataframe_subsidy_rate.reset_index(inplace=True)
# drop index column
agent_dataframe_subsidy_rate.drop(columns=['index'], inplace=True)


C:\Users\canan\AppData\Local\Temp\ipykernel_20300\2155249906.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  agent_dataframe_subsidy_rate = pd.concat(concatenated_dfs2.values(), keys=concatenated_dfs2.keys(), names=['replication/seed', 'subsidy_rate', 'index'])


In [44]:
# save the results to csv files
model_dataframe_subsidy_rate.to_csv(f"../results/{'model_sensitivity_results_subsidy_rate'}.csv", index=False)
agent_dataframe_subsidy_rate.to_csv(f"../results/{'agent_sensitivity_results_subsidy_rate'}.csv", index=False)

In [47]:
# upload the data with the following code
# change the file name to the file you want to load
# model_dataframe_subsidy_rate = pd.read_csv(f"../results/{'model_sensitivity_results_subsidy_rate'}.csv")

**4. Flood Probabilities**

In [8]:
# use defaultdict to store the results with different seeds
model_results_flood_prob =  defaultdict(list)
agent_results_flood_prob = defaultdict(list)
# Run the sensitivity for different flood prob, no subsidy included
for prob in flood_probabilities:
    for i in range(replication_number):
        seed = i  # set the seed
        model = AdaptationModel(number_of_households = number_of_households,
                                harvey_probability=prob, subsidy_rate=0,
                                  income_threshold=2000, saving_threshold=0.25, 
                                flood_map_choice="harvey", network="no_network", seed=seed)
        
        for tick in range(run_length):
            model.step() 
        model_data5 = model.datacollector.get_model_vars_dataframe()
        agent_data5 = model.datacollector.get_agent_vars_dataframe()
        agent_data5.reset_index(inplace=True)
        model_results_flood_prob[i,prob].append(model_data5)
        agent_results_flood_prob[i,prob].append(agent_data5)

**Save the Results**

In [9]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs = {}

# Concatenate the dataframes for each (i, rate) key
for key, result_list in model_results_flood_prob.items():
    i, prob = key
    concatenated_dfs[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
model_dataframe_flood_prob = pd.concat(concatenated_dfs.values(), keys=concatenated_dfs.keys(), names=['replication/seed', 'flood_probability', 'Step'])

# Reset the index 
model_dataframe_flood_prob.reset_index(inplace=True)



In [11]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs2 = {}

# Concatenate the dataframes for each (i, rate) key
for key, result_list in agent_results_flood_prob.items():
    i, prob = key
    concatenated_dfs2[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
agent_dataframe_flood_prob = pd.concat(concatenated_dfs2.values(), keys=concatenated_dfs2.keys(), names=['replication/seed', 'flood_probability', 'index'])

# Reset the index 
agent_dataframe_flood_prob.reset_index(inplace=True)
# drop index column
agent_dataframe_flood_prob.drop(columns=['index'], inplace=True)


C:\Users\canan\AppData\Local\Temp\ipykernel_23604\2904118766.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  agent_dataframe_flood_prob = pd.concat(concatenated_dfs2.values(), keys=concatenated_dfs2.keys(), names=['replication/seed', 'flood_probability', 'index'])


In [13]:
# save the results to csv files
model_dataframe_flood_prob.to_csv(f"../results/{'model_sensitivity_results_flood_prob'}.csv", index=False)
agent_dataframe_flood_prob.to_csv(f"../results/{'agent_sensitivity_results_flood_prob'}.csv", index=False)

**Experiment**

According to the results above you can check/change the design below! You can come up with better parameters
 **(delete this comment)**!!!1 you can also remove/add some of the parameters from the lists I created !!!!!!!1

In [53]:
# Experimental design
flood_probability_experiment = [0.02, 0.07, 0.1, 0.3, 0.5]  # different flood probabilities
subsidy_rate_experiment = [0, 0.1, 0.3]   
income_threshold_experiment = [1000, 5000, 8000]

In [55]:
# use defaultdict to store the results with different seeds
model_results_experiment =  defaultdict(list)
agent_results_experiment = defaultdict(list)
# Run the model
for prob in flood_probability_experiment: 
    for subsidy in subsidy_rate_experiment:
        for income in income_threshold_experiment:
            for i in range(replication_number):
                seed = i  # set the seed
                model = AdaptationModel(number_of_households = number_of_households,
                                            subsidy_rate=subsidy, 
                                            income_threshold=income, 
                                            saving_threshold=0.25, 
                                            harvey_probability=prob,
                                            flood_map_choice="harvey", network="no_network", seed=seed)
                for tick in range(run_length):
                    model.step() 

                model_data6 = model.datacollector.get_model_vars_dataframe()
                agent_data6 = model.datacollector.get_agent_vars_dataframe()
                agent_data6.reset_index(inplace=True)
                model_results_experiment[i,prob,subsidy,income].append(model_data6)
                agent_results_experiment[i,prob,subsidy,income].append(agent_data6)

**Save the Results**

In [61]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs = {}

# Concatenate the dataframes for each (i, subsidy, income, rate) key
for key, result_list in model_results_experiment.items():
    i, prob, subsidy, income = key
    concatenated_dfs[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
model_dataframe_experiment = pd.concat(concatenated_dfs.values(), keys=concatenated_dfs.keys(), names=['replication/seed','flood_probability','subsidy_rate', 'income_threshold', 'Step'])

# Reset the index
model_dataframe_experiment.reset_index(inplace=True)



In [63]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs2 = {}

# Concatenate the dataframes for each (i, subsidy, income, rate) key    
for key, result_list in agent_results_experiment.items():
    i, prob, subsidy, income = key
    concatenated_dfs2[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
agent_dataframe_experiment = pd.concat(concatenated_dfs2.values(), keys=concatenated_dfs2.keys(), names=['replication/seed', 'flood_probability', 'subsidy_rate', 'income_threshold', 'index'])

# Reset the index
agent_dataframe_experiment.reset_index(inplace=True)
# drop index column
agent_dataframe_experiment.drop(columns=['index'], inplace=True)


In [65]:
# save the results to csv files
model_dataframe_experiment.to_csv(f"../results/{'model_experiment_results'}.csv", index=False)
agent_dataframe_experiment.to_csv(f"../results/{'agent_experiment_results'}.csv", index=False)

In [66]:
# upload the data with the following code
# change the file name to the file you want to load
# model_dataframe_experiment = pd.read_csv(f"../results/{'model_experiment_results'}.csv")
# agent_dataframe_experiment = pd.read_csv(f"../results/{'agent_experiment_results'}.csv")